# Assignment 1
Welcome to the first assignment! Here you will implement User-User Collaborative Filtering. We will use the MovieLens 20M dataset.

You will write and execute your code in Python using this Jupyter Notebook.

**TASK:** Your job is to *fill in the missing code* only. The place to enter your code is clearly marked with comments. Please take care to **not** modify any other cells when using VisualStudioCode! This may lead to a negative impact on your grading. 

**SUBMISSION:** You will submit this Notebook via the Interface of JupyterLab.

- Submissions are possible until **01.04.25 23:59 CEST**.
- Do **NOT** rename the file it needs to be named as "assignment1.ipynb" (in the case if you want to run the Jupyter Notebook offline).
- Please **save** ("File -> "Save and Checkpoint") and **close** your Jupyter Notebook ("file" -> "Close and Halt") before you hand in your solution.
- Before handing in check if your Jupyter Notebook **validates**! 

**GRADING:** We will test whether your code produces the expected output. Therefore hidden tests will compare results of the standard solution with yours (based on the whole dataset - multiple, randomly selected inputs - accuracy of the solution must be within two decimal places). Note that the visible test cells are only an indicator for the correctness of your solution. They **do not** guarantee that your solution is correct. 

**Late submissions are not possible. Only submitted assignments will be collected by us at the time of the deadline!**

We reserve the right to carry out automatic plagiarism checks. Please do not exploit the submission system. We will look at all submissions and such submissions will be scored 0 points.

Note that there is a variable `DEBUG` set to `True` for debugging/testing purposes. This creates a small, more manageable subset of the data to work with.

## Preparation

### Import modules
Import python modules we will be using. Generally you should not import additional modules.

In [1]:
import csv
import pandas as pd
import numpy as np
from scipy import sparse as sp
from scipy.sparse.linalg import norm
import sklearn.preprocessing as pp

#if you wish to disable warnings, uncomment the following two lines
#import warnings
#warnings.filterwarnings("ignore")

Set some formatting options.

In [2]:
np.set_printoptions(threshold=500, precision=4)
pd.options.display.max_seq_items = 100
%precision 4

%load_ext autoreload
%autoreload 2

### Read the data
Load data from csv file `ratings.csv`. The result is stored in a `pandas.DataFrame` called `ratings_raw`.

In [3]:
data_location = '~/shared/194.035-2025S/data/ml-20m/ratings.csv' ## location of ratings.csv

In [4]:
ratings_raw = pd.read_csv(data_location)

### Truncate the data

For debugging, let's work with a *truncated* version of the data, containing up to 10000 users and 1000 movies. 

In [5]:
DEBUG = True

In [6]:
# Hidden

In [7]:
if DEBUG: 
    ratings_raw = ratings_raw[ (ratings_raw['userId'] < 10000) & (ratings_raw['movieId'] < 1000) ]

Let's see how the data looks like.

In [8]:
display(ratings_raw.head())

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


### Preprocess the data
Make sure that movies and users have consecutive indexes starting from 0. Also drop the timestamp column.

The resulting "cleaned" data are stored in `ratings`.

In [9]:
movieIds = ratings_raw.movieId.unique()
movieIds.sort()
userIds = ratings_raw.userId.unique()
userIds.sort()

m = userIds.size
n = movieIds.size
numRatings = len(ratings_raw)

print ("There are", m, "users,", n, "items and", numRatings, "ratings.")


## create internal ids for movies and users, that have consecutive indexes starting from 0
movieId_to_movieIDX = dict(zip(movieIds, range(0, movieIds.size)))
movieIDX_to_movieId = dict(zip(range(0, movieIds.size), movieIds))

userId_to_userIDX = dict(zip(userIds, range(0, userIds.size )))
userIDX_to_userId = dict(zip(range(0, userIds.size), userIds))

## drop timestamps
ratings = pd.concat([ratings_raw['userId'].map(userId_to_userIDX), ratings_raw['movieId'].map(movieId_to_movieIDX), ratings_raw['rating']], axis=1)
ratings.columns = ['user', 'item', 'rating']

display(ratings.head())

There are 9924 users, 968 items and 394638 ratings.


,user,item,rating
0,0,1,3.5
1,0,28,3.5
2,0,31,3.5
3,0,46,3.5
4,0,49,3.5


### Create the Ratings Matrix

We will convert the `ratings` `DataFrame` into a **Ratings Matrix**. Because it is very sparse, we will use the `scipy.sparse` module to efficiently store and access it.

Specifically, we will create two versions of the same ratings matrix:
- `R` is our basic matrix and is optimized for dot products, which will be useful when computing user-user similarities; `R` is stored in the Compressed Sparse Row format (`csr_matrix`).
- `R_dok` is a different view of the ratings matrix, which allows to quickly test whether a user-item rating exists; `R_dok` is stored in the Dictionary Of Keys format (`dok_matrix`).

In [10]:
R = sp.csr_matrix((ratings.rating, (ratings.user, ratings.item)))
R_dok = R.todok()

In [11]:
m = R.shape[0]
n = R.shape[1]
numRatings = R.count_nonzero()

print("There are", m, "users,", n, "items and", numRatings, "ratings.")

There are 9924 users, 968 items and 394638 ratings.


## The fun starts here!

### User Average Ratings

The following code computes the average rating of each user. This will be used for mean-centering, i.e., when computing similarities, as well as for making predictions.

In [12]:
user_sums = R.sum(axis=1).A1 ## matrix converted to 1-D array via .A1
user_cnts = (R != 0).sum(axis=1).A1
user_avgs = user_sums / user_cnts
print("user_avgs", user_avgs)

user_avgs [3.6667 4.2857 4.0588 ... 3.3441 3.2368 4.2333]


### User-User Similarity --- TO EDIT

The following function computes the mean-centered cosine similarity between two users.

*Tricks* that might be useful:

To subtract a scalar value `y` from all nonzero entries of a sparse vector `x`, do:
```
x.data = x.data - y
```

The dot product of a sparse vector `x` to sparse vector `y` is:
```
x.dot(y.T)
```

The norm of a sparse vector `x` is:
```
norm(x)
```


If a sparse vector `x` has only a single item, you can access it by:
```
x.toarray().item()
```

Note that `x.toarray()` returns the dense ndarray representation of sparse vector `x`.

In [13]:
def compute_pairwise_user_similarity(u_id, v_id):
    """
    This function takes two user IDs as an input and computes their pairwise similarity.

    :return: mean-centered cosine similarity score
    :rtype: float
    """
    u = R[u_id,:].copy()
    v = R[v_id,:].copy()
    
    # YOUR CODE HERE
    u.data = u.data - user_avgs[u_id]
    v.data = v.data - user_avgs[v_id]

    numerator = u.dot(v.T).toarray().item()
    denominator = norm(u) * norm(v)

    
    if denominator == 0:
        similarity = 0.;
    else:
        similarity = numerator/denominator
    
    return similarity

In [14]:
if DEBUG:
    display(compute_pairwise_user_similarity(2, 6))

0.0359

**DEBUG:** For the truncated dataset, the following should output sth like `0.0359`.

In [15]:
# Hidden

In [16]:
# Hidden

### User to all Users Similarities --- TO EDIT

The following function computes the mean-centered cosine similarities of a given user to all other users.

You should use the `compute_pairwise_user_similarity` function defined above.


You will get a **bonus point**, if you can avoid the for loop and not invoke `compute_pairwise_user_similarity`. The idea is to obtain a copy, say `R_copy`, of matrix `R` that has its rows mean-centered and normalized. This way the given user can be represented by a mean-centered and normalized vector `u`. Then, to obtain the similarity of the user to all others, one needs to take the dot product `R_copy.dot(u.T)`. 
**Note that the bonus point is only given, if the solution is efficient. This means that the function terminates in under 3 seconds on the server.**

In [17]:
def compute_user_similarities(u_id, R=R, user_avgs=user_avgs, user_cnts=user_cnts):
    """
    This function takes a user ID as an input and computes the pairwise similarity to all other users.
    
    :return: Similarity vector of the user to all others (shape: (m, ))
    :rtype: numpy.ndarray
    """
    uU = np.empty((m,))

    # YOUR CODE HERE
    R_copy = R.copy()
    R_copy.data = R_copy.data - np.repeat(user_avgs, user_cnts)

    u = R_copy[u_id, :].copy()
    u_norm = norm(u)

    # Avoid division by zero: handle cases where the norm is zero
    if u_norm == 0:
        return np.zeros(m)  # Proper indentation here

    row_norms = np.sqrt(R_copy.multiply(R_copy).sum(axis=1).A1)
    row_norms[row_norms == 0] = 1  # Avoid division by zero for rows with zero norm

    uU = R_copy.dot(u.T).toarray().flatten() / (u_norm * row_norms)
    return uU


In [18]:
if DEBUG:
    uU = compute_user_similarities(2)
    display(uU[6])

0.0359

**DEBUG:** For the truncated dataset, the following should again output `0.0359`.

In [19]:
# Hidden

In [20]:
# Hidden

In [21]:
# Hidden

### Create User Neighborhood --- TO EDIT

The following function creates the user neighborhood of a given user. It takes as input, the target user `u_id` and the target item`i_id`, and uses additional parameters, the size `k` of the neighborhood, and a flag `with_abs_sim`.

If `with_abs_sim` is `True`, the neighborhood should contain up to `k` users with the highest absolute similarity to the target user `u_id`.

If `with_abs_sim` is `False`, the neighborhood should contain up to `k` users with the highest similarity to the target user `u_id`.

The output of the function is `nh`, a `Dictionary` containing key-value entries of the form `v_id : sim(u_id, v_id)`, where `v_id` is another user and `sim(u_id, v_id)` is the similarity between `u_id` and `v_id`.

**Note:** The neighborhood of the target user should not contain itself, i.e., `u_id`, and only include users that have rated the target item `i_id`.


*Tricks* that might be useful:

`np.absolute(x)` returns an array containing the absolute values of each element in array `x`.

`np.argsort(x)` returns an array with the indices that sort array `x` in *increasing* order.

`x[::-1]` returns the reversed array of `x`. So, `np.argsort(x)[::-1]` contains the indices that sort x in *decreasing* order.

To check if user `u_id` has rated item `i_id`, the `R_dok` view of the ratings matrix is helpful:
```
(u_id, i_id) in R_dok
```

In [22]:
k = 5
with_abs_sim = False

def create_user_neighborhood(u_id, i_id, compute_user_similarities=compute_user_similarities, 
                             R=R, R_dok=R_dok, user_avgs=user_avgs, user_cnts=user_cnts, 
                             with_abs_sim=with_abs_sim, k=k):
    """
    This function takes a user ID and an item ID as input and calculates the neighborhood of size k.
    
    :return: neighborhood dictionary (user id: similarity) with k entries
    :rtype: dict[int,float]
    """
    nh = {}  # The neighborhood dict with (user id: similarity) entries
    # nh should not contain u_id and only include users that have rated i_id; there should be at most k neighbors
    uU = compute_user_similarities(u_id, R, user_avgs, user_cnts)
    uU_copy = uU.copy()  # So that we can modify it, but also keep the original
    
    # Filter out users who have not rated the target item i_id
    valid_users = [v_id for v_id in range(len(uU)) if (v_id != u_id) and ((v_id, i_id) in R_dok)]

    # Sort users by similarity
    if with_abs_sim:
        sorted_users = np.argsort(np.absolute(uU[valid_users]))[::-1]  # Sort by absolute similarity
    else:
        sorted_users = np.argsort(uU[valid_users])[::-1]  # Sort by similarity

    # Select the top-k users
    top_k_users = [valid_users[idx] for idx in sorted_users[:k]]

    # Create the neighborhood dictionary
    nh = {v_id: uU[v_id] for v_id in top_k_users}
    
    return nh


In [23]:
if DEBUG:
    k = 5
    with_abs_sim = False
    nh = create_user_neighborhood(0, 8,compute_user_similarities, R, R_dok, user_avgs, user_cnts, with_abs_sim, k)
    print("with_abs_sim", with_abs_sim)
    display(nh)
    with_abs_sim = True
    nh = create_user_neighborhood(0, 8,compute_user_similarities, R, R_dok, user_avgs, user_cnts, with_abs_sim, k)
    print("with_abs_sim", with_abs_sim)
    display(nh)

with_abs_sim False


{3439: 0.2213, 567: 0.2158, 8084: 0.1909, 7883: 0.1826, 5352: 0.1798}

with_abs_sim True


{4850: -0.2274, 3439: 0.2213, 567: 0.2158, 8084: 0.1909, 7883: 0.1826}

**DEBUG:** For the truncated dataset, the following should output sth like:
```
with_abs_sim False
{567: 0.2158, 3439: 0.2213, 5352: 0.1798, 7883: 0.1826, 8084: 0.1909}
with_abs_sim True
{567: 0.2158, 3439: 0.2213, 4850: -0.2274, 7883: 0.1826, 8084: 0.1909}
```

In [24]:
# Hidden

In [25]:
# Hidden

### Predict a Rating --- TO EDIT

The following function predicts the rating user `u_id` would give to item `i_id`. It uses the flag `with_deviations` to make the prediction.

If `with_deviations` is `True`, the prediction is made over *rating deviations*:
$$ s(u,i) = \overline{r_u} + \frac{\sum_{v \in N(u;i)}w_{uv} (r_{vi}-\overline{r_v})}{\sum_{v \in N(u;i)} |w_{uv}|} .$$

If `with_deviations` is `False`, the prediction is made directly over ratings:
$$ s(u,i) = \frac{\sum_{v \in N(u;i)}w_{uv} r_{vi}}{\sum_{v \in N(u;i)} |w_{uv}|} .$$

The output of the function is the predicted rating `prediction`.

In [26]:
with_deviations = True

def predict_rating(u_id, i_id, create_user_neighborhood=create_user_neighborhood, compute_user_similarities=compute_user_similarities):
    """
    This function takes a user ID and an item ID as an input and predicts the rating a user would give this item.
    
    :return: predicted rating for the selected item
    :rtype: float
    """
    
    if (u_id, i_id) in R_dok:
        print("user", u_id, "has rated item", i_id, "with", R[u_id, i_id])
    else:
        print("user", u_id, "has not rated item", i_id)
    print("k:", k, "with_deviations:", with_deviations, "with_abs_sim:", with_abs_sim)
    
    nh = create_user_neighborhood(u_id, i_id, compute_user_similarities, R, R_dok, user_avgs, user_cnts, with_abs_sim, k)
    
    neighborhood_weighted_avg = 0.

    # Create the neighborhood
    nh = create_user_neighborhood(u_id, i_id, compute_user_similarities, R, R_dok, user_avgs, user_cnts, with_abs_sim, k)

    # Compute the weighted average
    numerator = 0
    denominator = 0

    for v_id, similarity in nh.items():
        if with_deviations:
            numerator += similarity * (R_dok[v_id, i_id] - user_avgs[v_id])
        else:
            numerator += similarity * R_dok[v_id, i_id]
        denominator += abs(similarity)

    # Handle the case where the denominator is zero
    if denominator == 0:
        neighborhood_weighted_avg = 0
    else:
        neighborhood_weighted_avg = numerator / denominator

    if with_deviations:
        prediction = user_avgs[u_id] + neighborhood_weighted_avg
        print(f'prediction {prediction:.4f} (user_avg {user_avgs[u_id]:.4f} offset {neighborhood_weighted_avg:.4f})')
    else:
        prediction = neighborhood_weighted_avg
        print(f'prediction {prediction:.4f} (user_avg {user_avgs[u_id]:.4f})')
        
    return prediction


In [27]:
if DEBUG:
    k = 50
    with_abs_sim = True
    with_deviations = False
    predict_rating(0, 8)
    with_deviations = True
    predict_rating(0, 8)

user 0 has not rated item 8
k: 50 with_deviations: False with_abs_sim: True
prediction 2.3544 (user_avg 3.6667)
user 0 has not rated item 8
k: 50 with_deviations: True with_abs_sim: True
prediction 3.3064 (user_avg 3.6667 offset -0.3603)


**DEBUG:** For the truncated dataset, the following should output sth like:
```
user 0 has not rated item 8
k: 50 with_deviations: False with_abs_sim: True
prediction 2.3544 (user_avg 3.6667)
user 0 has not rated item 8
k: 50 with_deviations: True with_abs_sim: True
prediction 3.3064 (user_avg 3.6667 offset -0.3603)
```

In [28]:
# Hidden

In [29]:
# Hidden

## Additional Tests

For this part, **use the full dataset**, and not the truncated one.

In [30]:
k = 50
with_deviations = True
with_abs_sim = True
predict_rating(0, 8)
%time predict_rating(22, 35)

print("done!")

user 0 has not rated item 8
k: 50 with_deviations: True with_abs_sim: True
prediction 3.3064 (user_avg 3.6667 offset -0.3603)
user 22 has not rated item 35
k: 50 with_deviations: True with_abs_sim: True
prediction 4.2583 (user_avg 4.0000 offset 0.2583)
CPU times: user 21.8 ms, sys: 249 μs, total: 22.1 ms
Wall time: 22.1 ms
done!


**Sample output:**
```
user 0 has not rated item 8
k: 50 with_deviations: True with_abs_sim: True
prediction 3.0427 (user_avg 3.7429 offset -0.7001)
user 22 has not rated item 35
k: 50 with_deviations: True with_abs_sim: True
prediction 4.2453 (user_avg 3.9412 offset 0.3042)
CPU times: user 157 ms, sys: 60.3 ms, total: 218 ms
Wall time: 217 ms
done!

```

In [31]:
k = 50
with_deviations = False
with_abs_sim = False
predict_rating(0, 12)
%time predict_rating(22, 35)

print("done!")

user 0 has not rated item 12
k: 50 with_deviations: False with_abs_sim: False
prediction 3.0061 (user_avg 3.6667)
user 22 has not rated item 35
k: 50 with_deviations: False with_abs_sim: False
prediction 3.8786 (user_avg 4.0000)
CPU times: user 21.4 ms, sys: 193 μs, total: 21.6 ms
Wall time: 21.6 ms
done!


**Sample output:**
```
user 0 has not rated item 12
k: 50 with_deviations: False with_abs_sim: False
prediction 3.2975 (user_avg 3.7429)
user 22 has not rated item 35
k: 50 with_deviations: False with_abs_sim: False
prediction 3.9419 (user_avg 3.9412)
CPU times: user 157 ms, sys: 60.2 ms, total: 217 ms
Wall time: 217 ms
done!

```

In [32]:
# Hidden

In [33]:
# feel free to use this field for additional tests

In [34]:
# feel free to use this field for additional tests

In [35]:
# feel free to use this field for additional tests

In [36]:
# feel free to use this field for additional tests

In [37]:
# feel free to use this field for additional tests